# Interactive Education/AI Prototype
Use the inputs below to generate a standards-based question and provide a response for evaluation.

In [1]:
import json
import genai
import widgets
import textwrap

In [2]:
# display the input widgets
display(widgets.input_standard)
display(widgets.input_theme)

Dropdown(description='Select standard:', options=(('CCSS.ELA-LITERACY.W.4.9', 'CCSS.ELA-LITERACY.W.4.9 - Draw …

Text(value='', description='Context theme:', placeholder='Enter theme here')

In [3]:
# confirm theme selection
q_content = {}
if widgets.input_theme.value != "":
    widgets.print_section("Standard", widgets.input_standard.value)
    widgets.print_section("Theme", widgets.input_theme.value)
    messages = genai.generate_question_messages(
        widgets.input_standard.value,
        widgets.input_theme.value
    )
    print("\nGetting question...")
    response = genai.get_model_response(messages)
    print("Ready.")
    q_content = json.loads(response["choices"][0]["message"]["content"])
    widgets.print_section("Context", q_content["context"])
    widgets.print_section("Question", q_content["question"])
    # widgets.print_section("Context", content["context"])

else:
    print ("Please enter a theme.")


Standard:
CCSS.ELA-LITERACY.W.4.9 - Draw evidence from literary or
informational texts to support analysis, reflection, and
research.

Theme:
baseball

Getting question...
Ready.

Context:
In this passage, we will learn about the history of
baseball. Baseball is a popular sport in the United States
and has a rich history. It was first played in the mid-19th
century and has since become known as America's pastime. The
game is played with two teams, each consisting of nine
players. The objective of the game is to score more runs
than the opposing team. Baseball is played on a field with
four bases: first base, second base, third base, and home
plate. The game is divided into innings, and each team takes
turns batting and fielding. The team that scores the most
runs after nine innings wins the game. Baseball has produced
many legendary players, such as Babe Ruth, Jackie Robinson,
and Derek Jeter.

Question:
Based on the passage, who are some legendary baseball
players?


In [4]:
display(widgets.input_answer)

Textarea(value='', description='Answer:', placeholder='Enter answer here')

In [12]:
a_content = {}
if widgets.input_answer.value != "":
    messages = genai.generate_evaluation_messages(
        widgets.input_standard.value,
        widgets.input_theme.value,
        q_content,
        widgets.input_answer.value
    )
    print("\nEvaluating answer...")
    response = genai.get_model_response(messages)
    print("Ready.")
    a_content = json.loads(response["choices"][0]["message"]["content"])
    widgets.print_section("Score", str(a_content["score"]))
    widgets.print_section("Detail", a_content["detail"])
    widgets.print_section("Commentary", a_content["commentary"])
    print("\nEvaluation Rubric:")
    for k,v in q_content["evaluation"].items():
        content = f"[{k}]: {v}"
        print(textwrap.fill(content, width=60))
    print("")

else:
    print ("Please provide and answer.")


Evaluating answer...
Ready.

Score:
1

Detail:
The student provides minimal or incorrect information about
the key elements of football.

Commentary:
The student's answer, 'Football is fun,' does not accurately
identify any key elements of football mentioned in the
context. The student's answer does not demonstrate an
understanding of teamwork, strategy, physical fitness, or
any other important aspects of football.

Evaluation Rubric:
[4]: The student accurately identifies multiple key elements
of football, such as teamwork, strategy, and physical
fitness.
[3]: The student identifies some key elements of football,
but may miss one or two important aspects.
[2]: The student identifies a few key elements of football,
but misses several important aspects.
[1]: The student provides minimal or incorrect information
about the key elements of football.
[0]: The student does not provide any information about the
key elements of football.



In [18]:
a_resp = {}
a_resp["standard"] = q_content["standard"]
a_resp["context"] = q_content["context"]
a_resp["question"] = q_content["question"]
a_resp["evaluation"] = q_content["evaluation"]

print(json.dumps(a_resp, indent=4))


{
    "standard": "CCSS.ELA-LITERACY.W.4.9 - Draw evidence from literary or informational texts to support analysis, reflection, and research.",
    "context": "Football is a popular sport played around the world. It is a team sport that involves two teams of eleven players each. The objective of the game is to score more points than the opposing team by advancing the ball into the opponent's end zone. The game is played on a rectangular field with goalposts at each end. Players can pass, kick, or run with the ball to move it towards the opponent's end zone. Football requires teamwork, strategy, and physical fitness. It is a sport that requires both skill and athleticism.",
    "question": "Based on the context, what are some key elements of football?",
    "evaluation": {
        "4": "The student accurately identifies multiple key elements of football, such as teamwork, strategy, and physical fitness.",
        "3": "The student identifies some key elements of football, but may miss 

In [11]:
messages = genai.generate_ai_answers_message(q_content, 4)
response = genai.get_model_response(messages, temperature=1)

In [24]:
messages = genai.generate_question_messages(
    widgets.input_standard.value,
    widgets.input_theme.value
)
    
print (json.dumps(messages, indent=4))


messages = genai.generate_evaluation_messages(
    widgets.input_standard.value,
    widgets.input_theme.value,
    q_content,
    widgets.input_answer.value
)

print (json.dumps(messages, indent=4))

messages = genai.generate_ai_answers_message(
    q_content,
    2
)

print (json.dumps(messages, indent=4))


[
    {
        "role": "system",
        "content": "\nUse the following step-by-step instructions to respond to user inputs. The user is a grade-school teacher and will provide a common core education standard and a theme. The standard and the theme will be in quotes, separated by a comma. You will provide the teacher with the information necessary to evaluate a student against the standard.\nProvide the response content in JSON format.\nStep 1 - Use the provided theme to randomly select a topic and generate context material about which a student could answer questions. Use the vocabulary and syntax of the standard's target grade level. Use the key \"context\" for this JSON object.\nStep 2 - Generate a free-response question that can be answered by reading the context that will test the common core standard. Use the vocabulary and syntax of the standard's target grade level. Use the key \"question\" for this JSON object.\nStep 3 - Generate an evaluation rubric based on the context. U